<a href="https://colab.research.google.com/github/RoetGer/decisions-under-uncertainty/blob/main/data_science_and_stochastic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cvxpy
!pip install cvxstoc

     |████████████████████████████████| 389kB 8.3MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1352868 sha256=08579b57f9a4c49654d29378087d66241cfc0eca03bf6e6f00c64240005a071d
  Stored in directory: /root/.cache/pip/wheels/0b/a8/e7/8f3ba91a39294d538a92db052fd1fcba1fca74a58c8b022026
Successfully built pymc


# Data Science and Stochastic Programming

In this notebook we explore, how stochastic programming can be used to incorporate uncertainty stemming from data science models into our decision making process.

Let us start by introducing cvxstoc, a Python package for solving stochastic convex optimization problems.

In [2]:
import cvxpy
import cvxstoc
import numpy as np
import pymc

from cvxstoc import NormalRandomVariable, expectation, prob
from cvxpy import Maximize, Problem
from cvxpy.expressions.variable import Variable

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Samples to be taken
num_samples = 100

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = NormalRandomVariable(mu, Sigma)
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(weights.T*returns, num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


What we are trying to solve here is a simplified portfolio allocation problem, where the goal is to find a weight vector which maximizes the return under some constraints. 

The main differences to a more classical approach is that we are not working with a fixed vector of returns, but we assume that the returns are following a Gaussian distribution (with mean mu and covariance Sigma).

A consequence of this choice is that we are not merely trying to maximize the weighted sum of the returns (= weights.T*returns), but an expectation of this weighted sum with respect to the uncertain returns.

Moreover, while the first three constraints are rather standard (none of the portfolio positions should exceed 30% of the overall portfolio, the weights should be non-negative, and the combined weights add up to one), the last one is different from a deterministic optimization problem. The last constraint restricts the probability of the optimal portfolio to exceed a loss of 50% to 5%, i.e. for 100 samples of the return vector, we would only expect to have 5 times a loss higher than 50% with the optimized weights.

In [4]:
probl.solve()

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

optimal
Optimal value: 0.022908523858114094
Optimal weights: [3.00000000e-01 1.00000000e-01 3.00000000e-01 3.00000000e-01
 1.57715256e-11]


While it is fairly straightforward to see how this approach can be integrated with a data science solution (i.e. the data science model provides mean and covariance estimates for the Gaussian distribution), it is rather limited in its usage with a model.

For example, if we are using a Bayesian model to obtain posterior predictive samples, utilize dropout with a deep learning model to generate samples, or simply not use one of the distributions currently supported by cvxstocm, we would not be able to solve the resulting optimization problem.

In order to simplify the work with more complex distribution, we have developed the following function

In [65]:
from itertools import count

import numpy as np
import pymc
from cvxstoc import RandomVariable


def EmpiricalRandomVariable(name, 
                            samples,
                            mean,
                            interpolate=False, 
                            lower=-np.inf, 
                            upper=np.inf,
                            method="bootstrap",
                            rng_gen=None):
    '''
    Create a pymc node whose distribution comes either from a 
    kernel smoothing density estimate or via boostrapping from 
    the provided samples.
    '''
    
    if interpolate:
      rv_pymc = pymc.stochastic_from_data(
          name=rv_name, 
          data=samples, 
          lower=lower, 
          upper=upper)
    else:
        nobs = samples.shape[0]

        def logp(value):
            return -np.log(nobs)

        if method == "bootstrap":           
            def random():
                if rng_gen:
                    ridx = rng_gen.randint(low=0, high=nobs, size=1)
                    print(ridx)
                else:
                    ridx = np.random.randint(low=0, high=nobs, size=1)
                
                return np.squeeze(samples[ridx])
        
        elif method == "rolling":
            idx_gen = (idx for idx in count(0))

            def random():
                idx = next(idx_gen)
                print(idx)
                return np.squeeze(samples[idx % nobs])
        
        else:
          raise ValueError(
              "Unknown method. Currently, 'bootstrap' and 'rolling' are supported")

        value = random() 
        dtype = type(value)
    
        rv_pymc = pymc.Stochastic(
            logp = logp,
            doc = "A node which samples from the provided dataset",
            name = name,
            parents = {},
            random = random,
            trace = True,
            dtype = dtype)
    
    metadata = {"mu": mean}
    
    return RandomVariable(rv=rv_pymc, metadata=metadata)

In [70]:
samples = NormalRandomVariable(mu, Sigma).sample(100)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


In [ ]:
# Samples to be taken
num_samples = 10

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = EmpiricalRandomVariable("EmpiricalRV", 
                                  samples,
                                  mean = mu,
                                  interpolate=False,
                                  method="bootstrap",
                                  rng_gen=np.random.RandomState(5))
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(weights.T*returns, num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

probl.solve()

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

We can extend the example by optimizing a mean-variance problem, but in addition to the uncertain returns, we also assume that the covariance matrix is uncertain.

---



In [7]:
cov_samples = []

for i in range(2):
  sig = np.random.randn(n, n)
  sig = sig.T.dot(sig)
  cov_samples.append(sig)

cov_samples = np.stack(cov_samples, axis=0)

In [8]:
cov_samples = [
    np.eye(n) + np.array(pymc.WishartCov("cov-mat", 5, Sigma).random()) 
      for _ in range(10)
]
cov_samples = np.stack(cov_samples, axis=0)

TODO: There seems to be some issue with quadform when it comes to stochastic programs.

In [52]:
from cvxpy import Minimize

# Samples to be taken
num_samples = 100

# Create problem data.
n = 5
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
returns = EmpiricalRandomVariable("EmpiricalReturns", 
                                  NormalRandomVariable(mu, Sigma).sample(100),
                                  mean = mu,
                                  interpolate=False)
returns2 = EmpiricalRandomVariable("EmpiricalReturns", 
                                  NormalRandomVariable(mu, Sigma).sample(100),
                                  mean = mu,
                                  interpolate=False)
cov_mats = EmpiricalRandomVariable("CovMats",
                                   cov_samples,
                                   mean = Sigma,
                                   interpolate=False)
#cov_mats = cov_samples[0] + np.eye(n)
alpha = -0.5
beta = 0.05

# Create the stochastic optimization problem.
weights = Variable(n)
probl = Problem(
    Maximize(expectation(
        weights.T*returns - cvxpy.sum(weights.T*cov_mats*weights),#cvxpy.quad_form(weights, cov_mats),# - 0.5*cvxpy.quad_form(weights, cov_samples[1]),
        #-  0.5*cvxpy.quad_form(weights, cov_samples[1]),
        num_samples=num_samples)),
    [
      cvxpy.max(weights) <= 0.3,
      weights >= 0, 
      weights.T*np.ones(n) == 1,
      prob(weights.T*returns <= alpha, num_samples=num_samples) <= beta
    ]
)

probl.solve(verbose=True)

print(probl.status)
print("Optimal value:", probl.value)
print("Optimal weights:", weights.value)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


DCPError: ignored

In [55]:
weights.T*cov_mats*weights

Expression(UNKNOWN, UNKNOWN, ())

In [61]:
np.linalg.eig(cov_mats)

(array([7.10404969, 4.56014072, 2.61043479, 0.00844693, 0.02005087]),
 array([[-9.83828349e-02,  6.81913286e-01,  5.57104073e-02,
          4.83932667e-01,  5.36675519e-01],
        [ 3.38493596e-01, -4.09862823e-01,  6.99038116e-01,
          3.70354433e-02,  4.76873821e-01],
        [-9.27422518e-01, -1.42683572e-01,  3.13265772e-01,
         -1.18552851e-01,  8.56659176e-02],
        [ 1.25019253e-01,  5.88055712e-01,  4.77614592e-01,
         -5.95121327e-01, -2.37224733e-01],
        [ 7.50733306e-04, -2.89588494e-02, -4.26603939e-01,
         -6.29454825e-01,  6.48811624e-01]]))

In [19]:
??expectation